In [47]:
import pandas as pd
from sklearn.model_selection  import train_test_split
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import LabelEncoder
from src.helpers.plotters import plot_corr_matrix
import statsmodels.api as sm

In [48]:
df = pd.read_csv(r'../cleaned/output_encoded.csv')
df.head()

,fuel_type,mileage [km],model,name,price [pln],seller_type,year,engine_capacity [cm3],horsepower [km],is_gearbox_automatic,region
0,178162.801039,62000,101613.193302,85505.357299,63000,90852.118289,2019,1368,145,0,80689.855237
1,178162.801039,56800,101613.193302,133323.671059,99000,90852.118289,2018,1368,165,0,128266.815789
2,178162.801039,59000,101613.193302,85505.357299,71000,90852.118289,2018,1368,165,0,215862.601274
3,178162.801039,80089,101613.193302,85505.357299,59500,90852.118289,2016,1368,164,0,128266.815789
4,178162.801039,33000,101613.193302,85505.357299,70000,265742.166860,2021,1368,145,0,104437.423222


In [49]:
Y = df['price [pln]']
X = df.drop('price [pln]', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [50]:
lr = LinearRegression()

for i in range(1, df.shape[1]):
    forward = SequentialFeatureSelector(
    lr,
    k_features=i,
    forward=True,
    verbose=1
    )

    backward = SequentialFeatureSelector(
    lr,
    k_features=i,
    forward=False,
    verbose=1
    )
    sf_forward = forward.fit(X_train, y_train)
    sf_backward = backward.fit(X_train, y_train)
    print(str(i) + " forward_k_score " + str(sf_forward.k_score_) + " backward_k_score " + str(sf_backward.k_score_))
    print(list(sf_forward.k_feature_names_))
    print(list(sf_backward.k_feature_names_))

Features: 2/2

1 forward_k_score 0.7700927183899855 backward_k_score 0.7700927183899855
['name']
['name']


Features: 2/3

2 forward_k_score 0.7875436945330059 backward_k_score 0.7875436945330059
['model', 'name']
['model', 'name']


Features: 2/4

...

8 forward_k_score 0.8014887440166225 backward_k_score 0.8014887440166225
['fuel_type', 'mileage [km]', 'model', 'name', 'year', 'engine_capacity [cm3]', 'horsepower [km]', 'region']
['fuel_type', 'mileage [km]', 'model', 'name', 'year', 'engine_capacity [cm3]', 'horsepower [km]', 'region']


Features: 2/10

9 forward_k_score 0.8015793046455759 backward_k_score 0.8015793046455759
['fuel_type', 'mileage [km]', 'model', 'name', 'year', 'engine_capacity [cm3]', 'horsepower [km]', 'is_gearbox_automatic', 'region']
['fuel_type', 'mileage [km]', 'model', 'name', 'year', 'engine_capacity [cm3]', 'horsepower [km]', 'is_gearbox_automatic', 'region']


Features: 10/10

10 forward_k_score 0.8015187122788303 backward_k_score 0.8015187122788303
['fuel_type', 'mileage [km]', 'model', 'name', 'seller_type', 'year', 'engine_capacity [cm3]', 'horsepower [km]', 'is_gearbox_automatic', 'region']
['fuel_type', 'mileage [km]', 'model', 'name', 'seller_type', 'year', 'engine_capacity [cm3]', 'horsepower [km]', 'is_gearbox_automatic', 'region']


In [51]:
regressor = sm.OLS(endog=y_train,exog=X_train[['fuel_type', 'mileage [km]', 'model', 'name', 'year', 'engine_capacity [cm3]', 'horsepower [km]', 'is_gearbox_automatic', 'region']]).fit()
regressor.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            price [pln]   R-squared (uncentered):                   0.855
Model:                            OLS   Adj. R-squared (uncentered):              0.855
Method:                 Least Squares   F-statistic:                              1606.
Date:                Mon, 27 Jan 2025   Prob (F-statistic):                        0.00
Time:                        21:25:28   Log-Likelihood:                         -32217.
No. Observations:                2456   AIC:                                  6.445e+04
Df Residuals:                    2447   BIC:                                  6.450e+04
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
fuel_type                -0.2317      0.049     -4.735      0.000      -0.328      -0.136
mileage [km]             -0.2712      0.030     -8.952      0.000      -0.331      -0.212
model                     0.2815      0.025     11.232      0.000       0.232       0.331
name                      3.6117      0.077     46.616      0.000       3.460       3.764
year                   -186.2243      6.925    -26.890      0.000    -199.805    -172.644
engine_capacity [cm3]   -15.9897      2.814     -5.682      0.000     -21.508     -10.472
horsepower [km]         289.7970     37.248      7.780      0.000     216.756     362.838
is_gearbox_automatic  -8266.2544   5875.626     -1.407      0.160   -1.98e+04    3255.459
region                    0.1955      0.051      3.831      0.000       0.095       0.296
==============================================================================
Omnibus:                     1473.017   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           352203.209
Skew:                           1.694   Prob(JB):                         0.00
Kurtosis:                      61.568   Cond. No.                     7.77e+05
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 7.77e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""